In [ ]:
!pip install PyPDF2 python-docx spacy wordcloud
!pip install nltk
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
from PyPDF2 import PdfReader
import spacy
reader = PdfReader("Resume_Latest.pdf")
text = ""
for i, page in enumerate(reader.pages):
    page_text = page.extract_text()
    print(f"Page {i+1}:\n{page_text}\n")
    text += page_text + "\n"
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

clean_tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
tokens = [token.text for token in doc]

entities = [(ent.text, ent.label_) for ent in doc.ents]
print("Entities:", entities)



Page 1:
Dhruv Poduval
/githubhttps://github.com/dhruvpoduval |/linkedinlinkedin.com/in/dhruv-poduval-02aab2261 |/envel⌢pe
dhruvpoduval@gmail.com
Education
PES University 2021-2025
B.Tech Computer Science Current GPA: 8.52/10
National Public School 2019-2021
12th% - 96.8%
Coursework
Courses: Data Structures and Applications, Design and Analysis of Algorithms, Machine Intelligence,
Database Management Systems, Data Analytics, Database Technologies, Object Oriented Design and
Analysis, Generic Programming in C++.
Awards: 2x recipient of the MRD Scholarship for placing in the top 20% at the end of the semester.
Skills
Languages : C/C++(intermediate), Python, JavaScript(basic), HTML/CSS(basic), Java(basic), MySQL, MongoDB
Tools : Git/GitHub, VS Code, Docker
Libraries and Frameworks : ReactJS, NodeJS, Express, Flask, REST API’s
Experience
VuNet Systems |Software Engineering Intern Jan – Aug 2025
•Engineered modular UI automation test suites using Selenium andPlaywright , validating critical 